In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/financial-product-recommendation-rag/data/raw/



application_train.csv  bureau.csv


In [ ]:
import pandas as pd
import numpy as np

project_path="/content/drive/MyDrive/financial-product-recommendation-rag/data/raw/"

application_df=pd.read_csv(project_path +"application_train.csv")
bureau_df=pd.read_csv(project_path + "bureau.csv")


In [ ]:
threshold =0.7 #limit

app_high_miss=application_df.isnull().mean()[application_df.isnull().mean() > threshold]
print("High missing column in application data:",app_high_miss)

bureau_high_miss=bureau_df.isnull().mean()[bureau_df.isnull().mean() > threshold]
print("High missing column in bureau data:",bureau_high_miss)

High missing column in application data: Series([], dtype: float64)
High missing column in bureau data: AMT_ANNUITY    0.714735
dtype: float64


Application Data : None

Bureau data : AMT_ANNUITY    0.714735

In [ ]:
info_threshold_min=0.2
info_threshold_max=0.7

app_info=application_df.isnull().mean()[
    (application_df.isnull().mean() >= info_threshold_min) & (application_df.isnull().mean() <= info_threshold_max)
]

print("Info missing column in application data:",app_info)

bureau_info = bureau_df.isnull().mean()[
    (bureau_df.isnull().mean() >= info_threshold_min) & (bureau_df.isnull().mean() <= info_threshold_max)
]

print("Info missing column in bureau data:",bureau_info)


Info missing column in application data: OWN_CAR_AGE                     0.659908
OCCUPATION_TYPE                 0.313455
EXT_SOURCE_1                    0.563811
APARTMENTS_AVG                  0.507497
BASEMENTAREA_AVG                0.585160
YEARS_BEGINEXPLUATATION_AVG     0.487810
YEARS_BUILD_AVG                 0.664978
COMMONAREA_AVG                  0.698723
ELEVATORS_AVG                   0.532960
ENTRANCES_AVG                   0.503488
FLOORSMAX_AVG                   0.497608
FLOORSMIN_AVG                   0.678486
LANDAREA_AVG                    0.593767
LIVINGAPARTMENTS_AVG            0.683550
LIVINGAREA_AVG                  0.501933
NONLIVINGAPARTMENTS_AVG         0.694330
NONLIVINGAREA_AVG               0.551792
APARTMENTS_MODE                 0.507497
BASEMENTAREA_MODE               0.585160
YEARS_BEGINEXPLUATATION_MODE    0.487810
YEARS_BUILD_MODE                0.664978
COMMONAREA_MODE                 0.698723
ELEVATORS_MODE                  0.532960
ENTRANCES_MODE  

In [ ]:
# Application data flags
for col in app_info.index:
    application_df[col + "_missing_flag"] = application_df[col].isnull().astype(int)

# Bureau data flags
for col in bureau_info.index:
    bureau_df[col + "_missing_flag"] = bureau_df[col].isnull().astype(int)


In [ ]:
application_df[[col + "_missing_flag" for col in app_info.index]].head()
bureau_df[[col + "_missing_flag" for col in bureau_info.index]].head()


,DAYS_ENDDATE_FACT_missing_flag,AMT_CREDIT_MAX_OVERDUE_missing_flag,AMT_CREDIT_SUM_LIMIT_missing_flag
0,0,1,1
1,1,1,1
2,1,1,1
3,1,1,1
4,1,0,1


In [ ]:
processed_path = "/content/drive/MyDrive/financial-product-recommendation-rag/data/processed/"

application_df.to_csv(processed_path + "application_preprocessed.csv", index=False)
bureau_df.to_csv(processed_path + "bureau_preprocessed.csv", index=False)


In [ ]:

low_missing_threshold = 0.2

app_low_missing = application_df.isnull().mean()[
    application_df.isnull().mean() > 0
]
app_low_missing = app_low_missing[app_low_missing <= low_missing_threshold]
print("Low missing columns in application data:\n", app_low_missing)

bureau_low_missing = bureau_df.isnull().mean()[
    bureau_df.isnull().mean() > 0
]
bureau_low_missing = bureau_low_missing[bureau_low_missing <= low_missing_threshold]
print("\nLow missing columns in bureau data:\n", bureau_low_missing)


# Rule:
# - Numerical columns → fill with median (robust to outliers)
# - Categorical columns → fill with mode (most common value)
# We do this for columns with low missing percentage only


#application data
for col in app_low_missing.index:
    if application_df[col].dtype in ['float64', 'int64']:
        application_df[col].fillna(application_df[col].median(), inplace=True)
    else:
        application_df[col].fillna(application_df[col].mode()[0], inplace=True)

# Bureau Data
for col in bureau_low_missing.index:
    if bureau_df[col].dtype in ['float64', 'int64']:
        bureau_df[col].fillna(bureau_df[col].median(), inplace=True)
    else:
        bureau_df[col].fillna(bureau_df[col].mode()[0], inplace=True)

print("\nLow missing values filled with median/mode")


Low missing columns in application data:
 AMT_ANNUITY                   0.000039
AMT_GOODS_PRICE               0.000904
NAME_TYPE_SUITE               0.004201
CNT_FAM_MEMBERS               0.000007
EXT_SOURCE_2                  0.002146
EXT_SOURCE_3                  0.198253
OBS_30_CNT_SOCIAL_CIRCLE      0.003320
DEF_30_CNT_SOCIAL_CIRCLE      0.003320
OBS_60_CNT_SOCIAL_CIRCLE      0.003320
DEF_60_CNT_SOCIAL_CIRCLE      0.003320
DAYS_LAST_PHONE_CHANGE        0.000003
AMT_REQ_CREDIT_BUREAU_HOUR    0.135016
AMT_REQ_CREDIT_BUREAU_DAY     0.135016
AMT_REQ_CREDIT_BUREAU_WEEK    0.135016
AMT_REQ_CREDIT_BUREAU_MON     0.135016
AMT_REQ_CREDIT_BUREAU_QRT     0.135016
AMT_REQ_CREDIT_BUREAU_YEAR    0.135016
dtype: float64

Low missing columns in bureau data:
 DAYS_CREDIT_ENDDATE    0.061496
AMT_CREDIT_SUM         0.000008
AMT_CREDIT_SUM_DEBT    0.150119
dtype: float64


/tmp/ipython-input-1806756538.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  application_df[col].fillna(application_df[col].median(), inplace=True)
/tmp/ipython-input-1806756538.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value


Low missing values filled with median/mode


In [ ]:
processed_path = "/content/drive/MyDrive/financial-product-recommendation-rag/data/processed/"


application_df.to_csv(processed_path + "application_final.csv", index=False)


bureau_df.to_csv(processed_path + "bureau_final.csv", index=False)

print(" Preprocessed datasets saved successfully!")


 Preprocessed datasets saved successfully!
